In [ ]:
import cv2
import pandas as pd
import numpy as np
df_1 = pd.read_csv("data/Slideseq_1_glomeruli_position.txt", sep="\t", header=0, index_col=0)
df_2 = pd.read_csv("data/Slideseq_2_glomeruli_position.txt", sep="\t", header=0, index_col=0)
df_1["AP"] = df_1["AP"] / 4000
df_2["AP"] = df_2["AP"] / 4000
df_1.sort_values("OR", inplace=True)
df_2.sort_values("OR", inplace=True)
print(df_1.head())
print(df_2.head())
ORs_1 = df_1.OR.unique()
ORs_2 = df_2.OR.unique()
shared_ORs = [x for x in ORs_1 if x in ORs_2]
coords_1 = df_1[df_1["OR"].isin(shared_ORs)][["AP", "DV"]].values
coords_2 = df_2[df_2["OR"].isin(shared_ORs)][["AP", "DV"]].values
H, _ = cv2.findHomography(coords_1, coords_2)
n = coords_2.shape[0]
new_coords_1 = np.zeros((n, 3))
for i in range(n):
    pt1 = np.array([coords_1[i,0], coords_1[i,1], 1])
    pt1 = pt1.reshape(3, 1)
    pt2 = np.dot(H, pt1)
    pt2 = pt2/pt2[2]
    new_coords_1[i,0] = pt2[0]
    new_coords_1[i,1] = pt2[1]
with open("transformed_coords_1to2.txt", "w") as f:
    for index, row in df_2.iterrows():
        print(row["OR"], row["AP"], row["DV"], 2, file=f, sep="\t")
    for index, row in df_1.iterrows():
        pt1 = np.array([row["AP"], row["DV"], 1])
        pt1 = pt1.reshape(3, 1)
        pt2 = np.dot(H, pt1)
        pt2 = pt2/pt2[2]
        print(row["OR"], pt2[0,0], pt2[1,0], 1, file=f, sep="\t")